<a href="https://colab.research.google.com/github/ryanleeallred/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/LS_DS_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):` 

You should only truly need these two values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [0]:
# Define a function to obtain the probability that a person is drunk after positive breathalyzer tests
def prob_drunk_given_positive(prob_drug_prior, false_positive_rate, number_of_tests):
    posterior_probability=prob_drug_prior # set the prior
    for test in range(0, number_of_tests):
        P_user = posterior_probability # prior belief
        P_non_user = 1 - P_user  # complement of the prior belief
        P_pos_given_user = 1 # the true positive rate
        P_pos_given_non_user = false_positive_rate # the false positive rate
        posterior_probability = (P_pos_given_user*P_user)/((P_pos_given_user*P_user) + (P_pos_given_non_user*P_non_user))
    return posterior_probability

In [13]:
# Run the first test
prob_drunk_given_positive(1/1000, .08, 1)
# The probability that a person is drunk after one positive breathalyzer test is equal to 1.23%

0.012357884330202669

In [14]:
# Second test
prob_drunk_given_positive(1/1000, .08, 2)
# The probability that a person is drunk after two positive breathalyzer tests is equal to 13.52%

0.13525210993291495

In [21]:
# How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?
# Create a while loop to run tests until the probability is greater than 95%
prob=0
test_no = 1 # first test
while prob<=0.95:
  prob = prob_drunk_given_positive(1/1000, .08, test_no) # get the probability
  test_no += 1 # iteration for next test
print("Number of tests:", test_no-1, "\nProbability:", prob)
# print "test_no-1" because the iteration operation is at the end of the loop and if printing only "test_no" it will print the number of test where the function stopped

Number of tests: 4 
Probability: 0.9606895076105054


In [0]:
# 4 positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit

### The difference between Bayesian and Frequentist statistics
The difference between Bayesian and Frequentist statistics is in how the probabilities are used. In Frequentist statistics we use the probability only to model a process described as a sample, while in Bayesian statistics we use probabilities to model more kinds of uncertainty.
In other words, in Frequentist statistics we use probabilities only based on samples that we work on, and we try to see if a certain unknown constant either lies in a range (confidence inteval) or not. In Bayesian statistics, we begin with a prior belief, or a probability distribution which we know before collecting any data, and then we update the prior probability with a new one that we received after collecting more data.

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!